In [ ]:
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer


# Get the experiment run context
run = Run.get_context()

# import dataset
train = pd.read_csv("train.csv")

# Separting features and lable
X = train.drop(['SalePrice','Id'],axis=1)
y = train['SalePrice']

# Changing datatypes from int to object
X = X.astype({'MSSubClass':'object','OverallQual':'object', 'OverallCond':'object','MoSold':'object','YrSold':'object' })

# Replacing object variables with values of Nan to none
for elem in X.select_dtypes(include='object'):
    X[elem].fillna('None',inplace=True)
    
# Changing datatypes from int to object
X = X.astype({'MSSubClass':'object','OverallQual':'object', 'OverallCond':'object','MoSold':'object','YrSold':'object' })

# Getting indicators from categorical variables

X = pd.get_dummies(X)

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123)

# Handling missing values 
imp = SimpleImputer(strategy="median")
X_train =pd.DataFrame(imp.fit_transform(X_train))
X_test =pd.DataFrame(imp.fit_transform(X_test))

algorithm = 'Random Forest'
run.log("Agorithm",algorithm)

max_depth = 10
run.log("max tree depth", np.float(max_depth))

n_estimators = 100
run.log("Num of trees", n_estimators)

# Train a random forest regressor
regr = RandomForestRegressor(max_depth=max_depth, n_estimators = 100, random_state=123)
model = regr.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Root mean square error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
run.log("Root Mean square error", np.float(rmse))

# mean square error
mse = mean_squared_error(y_test, y_pred)
run.log("Mean square error", np.float(mse))

# Coefficient of determination
r2 = model.score(X_train, y_train)
run.log("R^2", np.float(r2))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/house_price_Forest_model.pkl')

run.complete()